# Implementatie van Lloyds algoritme

- Auteur: Jimmy Bierenbroodspot
- Datum: 12 juni 2024
- Locatie: Hogeschool Utrecht

Omdat het originele artikel waarin Lloyds algoritme gepubliceerd is niet over
computerwetenschap gaat maar over de elektrotechniek staat er niet echt een
duidelijk stuk pseudocode in het artikel. Om deze reden gebruiken we een ander
artikel waar Lloyds algoritme ook in beschreven wordt, namelijk Kmeans++
<sup>\[[1](#src2007)\]</sup>. Hierin worden de stappen als volgt beschreven:

1. Kies een willekeurig aantal clustercentra.
2. Voor elk clustercentrum, laat alle punten in de dataset wijzen naar het 
dichtstbijzijnde clustercentrum.
3. Verander elk clustercentrum naar het zwaartepunt van alle datapunten die naar
dit clustercentrum wijzen.
4. Herhaal stappen 2 en 3 totdat er niks meer veranderd.

## Stap 1

We beginnen met de juiste libraries te importeren. We gebruiken `numpy` vanwege
de efficiënte en snelle datastructuren. `numpy.typing` gebruiken we om
`numpy` datatypes te annoteren.

In [2]:
import numpy as np
import numpy.typing as npt

De eerste stap luidt dat we een aantal willekeurige clustercentra moeten kiezen.
Eerst nemen we nemen aan dat er een matrix (of 2-dimensionale lijst) mee wordt
gegeven en het aantal clusters dat we willen. met `replace=False` geven we aan
dat we geen duplicaten willen.

In [3]:
def choose_random_centers(dataset: npt.ArrayLike, num_clusters: int) -> npt.NDArray:
    return dataset[np.random.choice(dataset.shape[0], num_clusters, replace=False)]

Hieronder zullen we een kleine dataset aanmaken en testen de aangemaakte functie
hierop.

In [23]:
test_set: npt.NDArray = np.array([
    [1, 4, 6, 1],
    [3, 5, 1, 4],
    [1, 5, 0, 1],
    [8, 1, 4, 6],
    [9, 9, 9, 9],
])

choose_random_centers(test_set, 2)

array([[8, 1, 4, 6],
       [3, 5, 1, 4]])

Als we deze functie meerdere keren draaien zien we dat er inderdaad twee
willekeurige arrays uit de dataset worden gekozen.

## Stap 2

De tweede stap hebben we beschreven als:

> Voor elk clustercentrum, laat alle punten in de dataset wijzen naar het 
dichtstbijzijnde clustercentrum.

Dit gaan we interpreteren als het aanmaken van een nieuwe lijst met evenveel
elementen als rijen in de dataset. Elk element is een nummer dat wijst naar
de index in een lijst van clustercentra van de dichtstbijzijnde clustercentrum.
Lloyd gebruikt de Euclidische afstand in zijn algoritme
<sup>\[[2](#src1982)\]</sup> dus dat zullen wij ook gebruiken.

In [5]:
def get_euclidean_distance[ArrT: npt.ArrayLike](vector_one: ArrT, vector_two: ArrT) -> npt.NDArray:
    return np.linalg.norm(vector_one - vector_two)

`numpy` heeft de functie `numpy.linalg.norm` die de Euclidische afstand voor een
n-dimensionale reeks kan berekenen. Hieronder berekenen we ook handmatig de
Euclidische afstand om dubbel te checken dat deze functie inderdaad doet wat we
willen.

In [6]:
print("Zelf berekende Euclidische afstand:\t", np.sqrt((1 - 3)**2 + (4 - 5)**2 + (6 - 1)**2 + (1 - 4)**2))
print("np.linalg.norm:\t\t\t\t", get_euclidean_distance(test_set[0], test_set[1]))

Zelf berekende Euclidische afstand:	 6.244997998398398
np.linalg.norm:				 6.244997998398398


Om te bepalen welk clustercentrum het dichtstbij een punt zit moeten we alle
afstanden in de dataset tegenover alle clustercentra berekenen. Hiervoor maken
we een functie die een matrix en een vector als argumenten neemt en de 
Euclidische afstand voor elke vector in de matrix tegenover de vector argument
berekent. Het resultaat wordt in een nieuwe reeks teruggegeven, waar de index
van elke afstand overeenkomt met de index van het element in de dataset die
gebruikt is voor het berekenen van die afstand.

In [7]:
def get_matrix_euclidean_distance[ArrT: npt.ArrayLike](matrix: ArrT, vector: ArrT) -> npt.NDArray:
    return np.array([get_euclidean_distance(arr, vector) for arr in matrix])

Hieronder itereren we over elke afstand en nemen we de element uit de dataset
met dezelfde index en zien we dat de elementen met de afstand 0.0 inderdaad
dezelfde elementen zijn.

In [25]:
for i, distance in enumerate(get_matrix_euclidean_distance(test_set, test_set[0])):
    print("De Euclidische afstand tussen", test_set[i], "en", test_set[0], "is:", distance)

De Euclidische afstand tussen [1 4 6 1] en [1 4 6 1] is: 0.0
De Euclidische afstand tussen [3 5 1 4] en [1 4 6 1] is: 6.244997998398398
De Euclidische afstand tussen [1 5 0 1] en [1 4 6 1] is: 6.082762530298219
De Euclidische afstand tussen [8 1 4 6] en [1 4 6 1] is: 9.327379053088816
De Euclidische afstand tussen [9 9 9 9] en [1 4 6 1] is: 12.727922061357855


Uiteindelijk maken we de functie beschreven in het begin van dit hoofdstuk. We
doen dit door over elke vector in de matrix te itereren, de afstand te berekenen
en de index van de kleinste afstand te berekenen.

In [9]:
def get_closest_cluster_center[ArrT: npt.ArrayLike](matrix: ArrT, centers: ArrT) -> npt.ArrayLike:
    return np.array([np.argmin(get_matrix_euclidean_distance(centers, vector)) for vector in matrix])

Om te testen of deze functie werkt kunnen we de eerder gemaakte functie
gebruiken om twee willekeurige clustercentra te kiezen.

In [30]:
test_centers: npt.NDArray = choose_random_centers(test_set, 2)
print("De gekozen clustercentra:")
for center in test_centers:
    print(center)

De gekozen clustercentra:
[1 5 0 1]
[9 9 9 9]


Uiteindelijk voeren we deze functie uit.

In [11]:
closest_centers: npt.NDArray = get_closest_cluster_center(test_set, test_centers)

for i, vector in enumerate(test_set):
    print(vector, "is het dichtstbij:", test_centers[closest_centers[i]])

[1 4 6 1] is het dichtstbij: [1 4 6 1]
[3 5 1 4] is het dichtstbij: [3 5 1 4]
[1 5 0 1] is het dichtstbij: [3 5 1 4]
[8 1 4 6] is het dichtstbij: [3 5 1 4]
[9 9 9 9] is het dichtstbij: [3 5 1 4]


## Stap 3

Voor de derde stap moeten we elk clustercentrum veranderen naar het zwaartepunt
van alle elementen die naar dit clustercentrum wijzen. Wat we hiermee bedoelen
is dat elk clustercentrum het gemiddelde van alle elementen die het dichtstbij
dat clustercentrum zit.

We doen dit door eerst een boolean mask te maken, oftewel een reeks aan `True`
of `False` waar dit `True` is als het element aan een bepaalde conditie voldoet.
Omdat de reeks met dichtstbijzijnde clustercentra even lang is als de dataset
kunnen we checken of de index de huidige index is en als we deze mask toepassen
op de dataset krijgen we alleen de elementen terug die het dichtstbij zitten.

In [12]:
def get_center_of_mass_for_cluster_center[ArrT: npt.ArrayLike](
    matrix: ArrT, closest_centers: ArrT, cluster_idx: int
) -> npt.NDArray:
    mask: npt.NDArray = closest_centers == cluster_idx
    return np.average(matrix[mask,:], axis=0)

De volgende functie doet dit voor een matrix met clustercentra.

In [13]:
def get_all_center_of_masses[ArrT: npt.ArrayLike](matrix: ArrT, closest_centers: ArrT, num_clusters: int) -> npt.NDArray:
    return np.array([
        get_center_of_mass_for_cluster_center(matrix, closest_centers, i)
        for i
        in range(num_clusters)
    ])

Het resultaat is als volgt:

In [31]:
new_centers: npt.NDArray = get_all_center_of_masses(test_set, closest_centers, 2)

print("Oude clustercentra:\n", test_centers)
print("Nieuwe clustercentra:\n", new_centers)

Oude clustercentra:
 [[1 5 0 1]
 [9 9 9 9]]
Nieuwe clustercentra:
 [[1.   4.   6.   1.  ]
 [5.25 5.   3.5  5.  ]]


## Stap 4

De laatste stap is de vorige twee stappen uitvoeren totdat de clustercentra niet
meer veranderen. We beginnen door stap 1 uit te voeren en de centra te kiezen.
Vervolgens voeren we stap 2 uit en berekenen we de reeks met welke centra het 
dichtstbij elk element in de matrix zitten.

De `while`-loop in deze functie is zeer complex maar kan gelezen worden als
volgt:

1. We maken de variabele `new_centers` aan en geven die de waarde van 
`get_all_center_of_masses(matrix, closest_centers, num_clusters)`. Dit is in
essentie stap 3.
2. Omdat we de walrus-operator (`:=`) gebruiken geeft alles binnen de haakjes
dezelfde waarde terug als de variabele die we hierbinnen aanmaken. We
vergelijken dus of de nieuwe centra gelijk zijn aan de oude centra.
3. Omdat beide reeksen `numpy` reeksen zijn wordt het resultaat van de vorige 
stap een `numpy` reeks met `True` of `False` op basis van de conditie. Met
`.all()` checken we of alle waarde in deze reeks `True` zijn.

Zo lang de twee reeksen niet hetzelfde zijn worden de oude reeksen aangepast
naar de nieuwe reeksen.

In [15]:
def run_lloyds(matrix: npt.NDArray, num_clusters: int) -> tuple[npt.NDArray, npt.NDArray]:
    cluster_centers: npt.NDArray = choose_random_centers(matrix, num_clusters)
    closest_centers: npt.NDArray = get_closest_cluster_center(matrix, cluster_centers)

    while not ((new_centers := get_all_center_of_masses(matrix, closest_centers, num_clusters)) == cluster_centers).all():
        cluster_centers = new_centers
        closest_centers = get_closest_cluster_center(matrix, cluster_centers)

    return cluster_centers, closest_centers

### Leuk feitje

Omdat alle functies die gebruikt worden voor dit algoritme een enkele regel zijn
is het mogelijk om Lloyds algoritme in 4 regels uit te voeren:

1. Voor het aanmaken van de eerste clustercentra, dit kan in een regel d.m.v. 
een komma.
2. De `while`-loop.
3. Het veranderen van de nieuwe variabelen kan ook op een regel.
4. De `return`-statement.

Dit zou natuurlijk onleesbaar zijn maar leuk voor code-golfing!

Uiteindelijk kunnen we de gemaakte algoritme uitvoeren:

In [16]:
run_lloyds(test_set, 2)

(array([[3.25, 3.75, 2.75, 3.  ],
        [9.  , 9.  , 9.  , 9.  ]]),
 array([0, 0, 0, 0, 1]))

## Vervolg

Het zou netjes zijn de code nog te testen maar de vorm van de algoritme zoals
het nu is, is nog niet schaalbaar dus we zouden hier nog wat aan willen doen.
Het algoritme zoals die zich in deze notebook bevind is er puur voor het
demonstreren hoe het algoritme is opgebouwd. Tests zullen elders uitgevoerd
worden.

## Sources

<a id="src2007"></a> \[1\] Arthur, D., & Vassilvitskii, S. (2007). k-means++: the advantages of careful seeding. Soda, 1027–1035. https://doi.org/10.5555/1283383.1283494

<a id="src1982"></a> \[2\] Lloyd, S. (1982). Least squares quantization in PCM. IEEE Transactions on Information Theory, 28(2), 129–137. https://doi.org/10.1109/tit.1982.1056489